# IMPORT BIBLIOTEK

In [5]:
import mysql.connector
import pandas as pd
import random

# POŁĄCZENIE Z BAZĄ DANYCH

In [8]:

con = mysql.connector.connect(
    host = "localhost",
    user = "testUser",
    password = "test123User",
    database = "test"
)

if(con):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

Połączenie udane


# TWORZENIE SCHEMATU BAZY DANYCH

In [22]:
cursor = con.cursor()

schema_sql = """
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    date_of_birth DATE,
    is_insured BOOLEAN,
    email VARCHAR(150),
    phone_number VARCHAR(30),
    emergency_contact_name VARCHAR(100),
    emergency_contact_phone VARCHAR(30),
    emergency_contact_relation VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Spacecrafts (
    vehicle_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    type VARCHAR(100),
    capacity INT,
    manufacturing_date DATE,
    status VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Employees (
    employee_id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    role VARCHAR(100),
    specialization VARCHAR(100),
    assigned_vehicle_id INT,
    FOREIGN KEY (assigned_vehicle_id) REFERENCES Spacecrafts(vehicle_id)
);

CREATE TABLE IF NOT EXISTS VehicleApprovals (
    approval_id INT AUTO_INCREMENT PRIMARY KEY,
    vehicle_id INT,
    employee_id INT,
    approval_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    note TEXT,
    FOREIGN KEY (vehicle_id) REFERENCES Spacecrafts(vehicle_id),
    FOREIGN KEY (employee_id) REFERENCES Employees(employee_id)
);

CREATE TABLE IF NOT EXISTS Destinations (
    destination_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    description TEXT,
    distance_range VARCHAR(100), -- zmieniono z 'range'
    x_coordinate FLOAT,
    y_coordinate FLOAT,
    z_coordinate FLOAT,
    gravity_level FLOAT,
    radiation_risk_level INT
);

CREATE TABLE IF NOT EXISTS TripTypes (
    trip_type_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    description TEXT,
    estimated_duration_days INT,
    is_one_way BOOLEAN
);

CREATE TABLE IF NOT EXISTS Trips (
    trip_id INT AUTO_INCREMENT PRIMARY KEY,
    trip_type_id INT,
    destination_id INT,
    launch_date TIMESTAMP,
    return_date TIMESTAMP,
    spacecraft_id INT,
    launch_method VARCHAR(100),
    status VARCHAR(50),
    organization_cost DECIMAL(12,2),
    FOREIGN KEY (trip_type_id) REFERENCES TripTypes(trip_type_id),
    FOREIGN KEY (destination_id) REFERENCES Destinations(destination_id),
    FOREIGN KEY (spacecraft_id) REFERENCES Spacecrafts(vehicle_id)
);

CREATE TABLE IF NOT EXISTS TripParticipants (
    participant_id INT AUTO_INCREMENT PRIMARY KEY,
    trip_id INT,
    customer_id INT,
    ticket_price DECIMAL(10,2),
    seat_number VARCHAR(10),
    FOREIGN KEY (trip_id) REFERENCES Trips(trip_id),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    amount DECIMAL(10,2),
    transaction_date TIMESTAMP,
    method VARCHAR(50),
    description TEXT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
"""
for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement + ';')

con.commit()
print("Schemat bazy danych został utworzony.")

Schemat bazy danych został utworzony.


In [23]:

# Wczytanie danych z plików CSV
imiona_men_df = pd.read_csv('data/imiona_men.csv')
nazwiska_men_df = pd.read_csv('data/nazwiska_men.csv')
imiona_woman_df = pd.read_csv('data/imiona_woman.csv')
nazwiska_woman_df = pd.read_csv('data/nazwiska_woman.csv')

# Zamiana na listy imion/nazwisk
imiona_men = imiona_men_df['IMIĘ_PIERWSZE'].dropna().unique().tolist()
nazwiska_men = nazwiska_men_df['Nazwisko aktualne'].dropna().unique().tolist()
imiona_woman = imiona_woman_df['IMIĘ_PIERWSZE'].dropna().unique().tolist()
nazwiska_woman = nazwiska_woman_df['Nazwisko aktualne'].dropna().unique().tolist()

# Losowanie 500 par dla każdej płci
sampled_imiona_men = random.sample(imiona_men, 500)
sampled_nazwiska_men = random.sample(nazwiska_men, 500)
sampled_imiona_woman = random.sample(imiona_woman, 500)
sampled_nazwiska_woman = random.sample(nazwiska_woman, 500)

# Tworzenie listy krotek (first_name, last_name)
pary_men = list(zip(sampled_imiona_men, sampled_nazwiska_men))
pary_woman = list(zip(sampled_imiona_woman, sampled_nazwiska_woman))

# Połączenie obu płci
wszystkie_pary = pary_men + pary_woman
random.shuffle(wszystkie_pary)  # Losowa kolejność

In [ ]:
cursor = con.cursor()
sql = "INSERT INTO Customers (first_name, last_name) VALUES (%s, %s)"
cursor.executemany(sql, wszystkie_pary)
con.commit()
print("Wstawiono 1000 klientów.")

ProgrammingError: Cursor is not connected